<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas
dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})

# Codificar las etiquetas
label_encoder = LabelEncoder()
dftrain['class'] = label_encoder.fit_transform(dftrain['class'])
dftrain

,image_filename,mask_filename,class
0,image_003.png,mask_003.png,0
1,image_004.png,mask_004.png,0
2,image_008.png,mask_008.png,0
3,image_012.png,mask_012.png,0
4,image_016.png,mask_016.png,0
...,...,...,...
332,image_399.png,mask_399.png,2
333,image_400.png,mask_400.png,2
334,image_416.png,mask_416.png,2
335,image_419.png,mask_419.png,2


In [45]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


In [46]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224, 224
n_channels = 3                # número de canales (RGB)
n_classes = 5                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = 'class'

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_str):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_str                                     # No es necesario convertir a número si las etiquetas son strings
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')

Número de lotes del conjunto de entrenamiento: 11
Número de lotes del conjunto de validación: 3


In [43]:
from tensorflow.data import Dataset
from tensorflow.keras import layers, models

# Definir el modelo de red neuronal convolucional (CNN)
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(n_classes, activation='softmax')
    ])
    return model

# Crear el modelo
input_shape = (img_width, img_height, n_channels)
model = create_model(input_shape)

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

Epoch 1/10


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
/content/train/image_359.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_9578]